In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# One run full walktrhough "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "* Do the full walk through on the large data set\n",
    "* Refactor the source code and bring it to individual scripts\n",
    "* Ensure a full run with one click"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Your base path is at: ads_covid-19'"
      ]
     },
     "execution_count": 1,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "## check some parameters\n",
    "## depending where you launch your notebook, the relative path might not work\n",
    "## you should start the notebook server from your base path\n",
    "## when opening the notebook, typically your path will be ../ads_covid-19/notebooks\n",
    "import os\n",
    "if os.path.split(os.getcwd())[-1]=='notebooks':\n",
    "    os.chdir(\"../\")\n",
    "\n",
    "'Your base path is at: '+os.path.split(os.getcwd())[-1]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1 Update all data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {
    "code_folding": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Error : b'From https://github.com/CSSEGISandData/COVID-19\\n   ca11cb6e..45909bb3  master     -> origin/master\\n   053211eb..e466d927  web-data   -> origin/web-data\\n'\n",
      "out : b'Updating ca11cb6e..45909bb3\\nFast-forward\\n csse_covid_19_data/README.md                       |   19 +-\\n .../csse_covid_19_daily_reports/04-21-2020.csv     | 3092 ++++++++++\\n .../csse_covid_19_daily_reports/04-22-2020.csv     | 3100 ++++++++++\\n .../csse_covid_19_daily_reports/04-23-2020.csv     | 3121 ++++++++++\\n .../csse_covid_19_daily_reports/04-24-2020.csv     | 3129 ++++++++++\\n .../csse_covid_19_daily_reports_us/04-21-2020.csv  |   60 +\\n .../csse_covid_19_daily_reports_us/04-22-2020.csv  |   60 +\\n .../csse_covid_19_daily_reports_us/04-23-2020.csv  |   60 +\\n .../csse_covid_19_daily_reports_us/04-24-2020.csv  |   60 +\\n .../csse_covid_19_time_series/Errata.csv           |  154 +-\\n .../time_series_covid19_confirmed_US.csv           | 6525 ++++++++++----------\\n .../time_series_covid19_confirmed_global.csv       |  530 +-\\n .../time_series_covid19_deaths_US.csv              | 6525 ++++++++++----------\\n .../time_series_covid19_deaths_global.csv          |  530 +-\\n .../time_series_covid19_recovered_global.csv       |  502 +-\\n 15 files changed, 20106 insertions(+), 7361 deletions(-)\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports/04-21-2020.csv\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports/04-22-2020.csv\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports/04-23-2020.csv\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports/04-24-2020.csv\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports_us/04-21-2020.csv\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports_us/04-22-2020.csv\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports_us/04-23-2020.csv\\n create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports_us/04-24-2020.csv\\n'\n",
      " Number of regions rows: 412\n"
     ]
    }
   ],
   "source": [
    "# %load src/data/get_data.py\n",
    "\n",
    "import subprocess\n",
    "import os\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "\n",
    "from datetime import datetime\n",
    "\n",
    "import requests\n",
    "import json\n",
    "\n",
    "def get_johns_hopkins():\n",
    "    ''' Get data by a git pull request, the source code has to be pulled first\n",
    "        Result is stored in the predifined csv structure\n",
    "    '''\n",
    "    git_pull = subprocess.Popen( \"/usr/bin/git pull\" ,\n",
    "                         cwd = os.path.dirname( 'data/raw/COVID-19/' ),\n",
    "                         shell = True,\n",
    "                         stdout = subprocess.PIPE,\n",
    "                         stderr = subprocess.PIPE )\n",
    "    (out, error) = git_pull.communicate()\n",
    "\n",
    "\n",
    "    print(\"Error : \" + str(error))\n",
    "    print(\"out : \" + str(out))\n",
    "\n",
    "\n",
    "def get_current_data_germany():\n",
    "    ''' Get current data from germany, attention API endpoint not too stable\n",
    "        Result data frame is stored as pd.DataFrame\n",
    "\n",
    "    '''\n",
    "    # 16 states\n",
    "    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')\n",
    "\n",
    "    # 400 regions / Landkreise\n",
    "    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')\n",
    "\n",
    "    json_object=json.loads(data.content)\n",
    "    full_list=[]\n",
    "    for pos,each_dict in enumerate (json_object['features'][:]):\n",
    "        full_list.append(each_dict['attributes'])\n",
    "\n",
    "    pd_full_list=pd.DataFrame(full_list)\n",
    "    pd_full_list.to_csv('data/raw/NPGEO/GER_state_data.csv',sep=';')\n",
    "    print(' Number of regions rows: '+str(pd_full_list.shape[0]))\n",
    "\n",
    "if __name__ == '__main__':\n",
    "    get_johns_hopkins()\n",
    "    get_current_data_germany()\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Process pipeline "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {
    "code_folding": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      " Number of rows stored: 24816\n",
      " Latest date is: 2020-04-24 00:00:00\n"
     ]
    }
   ],
   "source": [
    "# %load src/data/process_JH_data.py\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "\n",
    "from datetime import datetime\n",
    "\n",
    "\n",
    "def store_relational_JH_data():\n",
    "    ''' Transformes the COVID data in a relational data set\n",
    "\n",
    "    '''\n",
    "\n",
    "    data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'\n",
    "    pd_raw=pd.read_csv(data_path)\n",
    "\n",
    "    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',\n",
    "                      'Province/State':'state'})\n",
    "\n",
    "    pd_data_base['state']=pd_data_base['state'].fillna('no')\n",
    "\n",
    "    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)\n",
    "\n",
    "\n",
    "    pd_relational_model=pd_data_base.set_index(['state','country']) \\\n",
    "                                .T                              \\\n",
    "                                .stack(level=[0,1])             \\\n",
    "                                .reset_index()                  \\\n",
    "                                .rename(columns={'level_0':'date',\n",
    "                                                   0:'confirmed'},\n",
    "                                                  )\n",
    "\n",
    "    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')\n",
    "\n",
    "    pd_relational_model.to_csv('data/processed/COVID_relational_confirmed.csv',sep=';',index=False)\n",
    "    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))\n",
    "    print(' Latest date is: '+str(max(pd_relational_model.date)))\n",
    "if __name__ == '__main__':\n",
    "\n",
    "    store_relational_JH_data()\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3  Filter and Doubling Rate Calculation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {
    "code_folding": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "the test slope is: [2.]\n",
      "            date state  country  confirmed  confirmed_filtered  confirmed_DR  \\\n",
      "13625 2020-04-20    no  Germany   147065.0            146906.0     78.000000   \n",
      "13626 2020-04-21    no  Germany   148291.0            148863.4     94.526338   \n",
      "13627 2020-04-22    no  Germany   150648.0            150826.4     82.985208   \n",
      "13628 2020-04-23    no  Germany   153129.0            152897.0     62.294061   \n",
      "13629 2020-04-24    no  Germany   154999.0            154967.6     70.294338   \n",
      "\n",
      "       confirmed_filtered_DR  \n",
      "13625              73.052980  \n",
      "13626              77.183388  \n",
      "13627              75.943917  \n",
      "13628              74.802790  \n",
      "13629              73.841882  \n"
     ]
    }
   ],
   "source": [
    "# %load src/features/build_features.py\n",
    "\n",
    "import numpy as np\n",
    "from sklearn import linear_model\n",
    "reg = linear_model.LinearRegression(fit_intercept=True)\n",
    "import pandas as pd\n",
    "\n",
    "from scipy import signal\n",
    "\n",
    "\n",
    "def get_doubling_time_via_regression(in_array):\n",
    "    ''' Use a linear regression to approximate the doubling rate\n",
    "\n",
    "        Parameters:\n",
    "        ----------\n",
    "        in_array : pandas.series\n",
    "\n",
    "        Returns:\n",
    "        ----------\n",
    "        Doubling rate: double\n",
    "    '''\n",
    "\n",
    "    y = np.array(in_array)\n",
    "    X = np.arange(-1,2).reshape(-1, 1)\n",
    "\n",
    "    assert len(in_array)==3\n",
    "    reg.fit(X,y)\n",
    "    intercept=reg.intercept_\n",
    "    slope=reg.coef_\n",
    "\n",
    "    return intercept/slope\n",
    "\n",
    "\n",
    "def savgol_filter(df_input,column='confirmed',window=5):\n",
    "    ''' Savgol Filter which can be used in groupby apply function (data structure kept)\n",
    "\n",
    "        parameters:\n",
    "        ----------\n",
    "        df_input : pandas.series\n",
    "        column : str\n",
    "        window : int\n",
    "            used data points to calculate the filter result\n",
    "\n",
    "        Returns:\n",
    "        ----------\n",
    "        df_result: pd.DataFrame\n",
    "            the index of the df_input has to be preserved in result\n",
    "    '''\n",
    "\n",
    "    degree=1\n",
    "    df_result=df_input\n",
    "\n",
    "    filter_in=df_input[column].fillna(0) # attention with the neutral element here\n",
    "\n",
    "    result=signal.savgol_filter(np.array(filter_in),\n",
    "                           window, # window size used for filtering\n",
    "                           1)\n",
    "    df_result[str(column+'_filtered')]=result\n",
    "    return df_result\n",
    "\n",
    "def rolling_reg(df_input,col='confirmed'):\n",
    "    ''' Rolling Regression to approximate the doubling time'\n",
    "\n",
    "        Parameters:\n",
    "        ----------\n",
    "        df_input: pd.DataFrame\n",
    "        col: str\n",
    "            defines the used column\n",
    "        Returns:\n",
    "        ----------\n",
    "        result: pd.DataFrame\n",
    "    '''\n",
    "    days_back=3\n",
    "    result=df_input[col].rolling(\n",
    "                window=days_back,\n",
    "                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)\n",
    "\n",
    "\n",
    "\n",
    "    return result\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "def calc_filtered_data(df_input,filter_on='confirmed'):\n",
    "    '''  Calculate savgol filter and return merged data frame\n",
    "\n",
    "        Parameters:\n",
    "        ----------\n",
    "        df_input: pd.DataFrame\n",
    "        filter_on: str\n",
    "            defines the used column\n",
    "        Returns:\n",
    "        ----------\n",
    "        df_output: pd.DataFrame\n",
    "            the result will be joined as a new column on the input data frame\n",
    "    '''\n",
    "\n",
    "    must_contain=set(['state','country',filter_on])\n",
    "    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'\n",
    "\n",
    "    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten\n",
    "\n",
    "    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()\n",
    "\n",
    "    #print('--+++ after group by apply')\n",
    "    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())\n",
    "\n",
    "    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')\n",
    "    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')\n",
    "    #print(df_output[df_output['country']=='Germany'].tail())\n",
    "    return df_output.copy()\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "def calc_doubling_rate(df_input,filter_on='confirmed'):\n",
    "    ''' Calculate approximated doubling rate and return merged data frame\n",
    "\n",
    "        Parameters:\n",
    "        ----------\n",
    "        df_input: pd.DataFrame\n",
    "        filter_on: str\n",
    "            defines the used column\n",
    "        Returns:\n",
    "        ----------\n",
    "        df_output: pd.DataFrame\n",
    "            the result will be joined as a new column on the input data frame\n",
    "    '''\n",
    "\n",
    "    must_contain=set(['state','country',filter_on])\n",
    "    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'\n",
    "\n",
    "\n",
    "    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()\n",
    "\n",
    "    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',\n",
    "                             'level_2':'index'})\n",
    "\n",
    "    #we do the merge on the index of our big table and on the index column after groupby\n",
    "    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')\n",
    "    df_output=df_output.drop(columns=['index'])\n",
    "\n",
    "\n",
    "    return df_output\n",
    "\n",
    "\n",
    "if __name__ == '__main__':\n",
    "    test_data_reg=np.array([2,4,6])\n",
    "    result=get_doubling_time_via_regression(test_data_reg)\n",
    "    print('the test slope is: '+str(result))\n",
    "\n",
    "    pd_JH_data=pd.read_csv('data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])\n",
    "    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()\n",
    "\n",
    "    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|\n",
    "    #                  (pd_JH_data['country']=='Germany'))]\n",
    "\n",
    "    pd_result_larg=calc_filtered_data(pd_JH_data)\n",
    "    pd_result_larg=calc_doubling_rate(pd_result_larg)\n",
    "    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')\n",
    "\n",
    "\n",
    "    mask=pd_result_larg['confirmed']>100\n",
    "    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)\n",
    "    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',sep=';',index=False)\n",
    "    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "            date state country  confirmed  confirmed_filtered  confirmed_DR  \\\n",
      "23589 2020-04-20    no      US   784326.0            785509.6     29.098736   \n",
      "23590 2020-04-21    no      US   811865.0            812904.2     29.591375   \n",
      "23591 2020-04-22    no      US   840351.0            842214.0     28.993509   \n",
      "23592 2020-04-23    no      US   869170.0            872150.9     29.332938   \n",
      "23593 2020-04-24    no      US   905358.0            902087.8     26.816384   \n",
      "\n",
      "       confirmed_filtered_DR  \n",
      "23589              26.573917  \n",
      "23590              28.285803  \n",
      "23591              28.694161  \n",
      "23592              28.437804  \n",
      "23593              29.132973  \n"
     ]
    }
   ],
   "source": [
    "print(pd_result_larg[pd_result_larg['country']=='US'].tail())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4 Visual Board"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "/Users/kienlef/SynologyDrive/PPT_slides_Vorlesung/2020_Video_Vorlesung/ads_covid-19\n",
      "Running on http://127.0.0.1:8050/\n",
      "Debugger PIN: 451-893-824\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      " * Tip: There are .env or .flaskenv files present. Do \"pip install python-dotenv\" to use them.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      " * Serving Flask app \"__main__\" (lazy loading)\n",
      " * Environment: production\n",
      "   WARNING: This is a development server. Do not use it in a production deployment.\n",
      "   Use a production WSGI server instead.\n",
      " * Debug mode: on\n"
     ]
    }
   ],
   "source": [
    "# %load src/visualization/visualize.py\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "\n",
    "import dash\n",
    "dash.__version__\n",
    "import dash_core_components as dcc\n",
    "import dash_html_components as html\n",
    "from dash.dependencies import Input, Output,State\n",
    "\n",
    "import plotly.graph_objects as go\n",
    "\n",
    "import os\n",
    "print(os.getcwd())\n",
    "df_input_large=pd.read_csv('data/processed/COVID_final_set.csv',sep=';')\n",
    "\n",
    "\n",
    "fig = go.Figure()\n",
    "\n",
    "app = dash.Dash()\n",
    "app.layout = html.Div([\n",
    "\n",
    "    dcc.Markdown('''\n",
    "    #  Applied Data Science on COVID-19 data\n",
    "\n",
    "    Goal of the project is to teach data science by applying a cross industry standard process,\n",
    "    it covers the full walkthrough of: automated data gathering, data transformations,\n",
    "    filtering and machine learning to approximating the doubling time, and\n",
    "    (static) deployment of responsive dashboard.\n",
    "\n",
    "    '''),\n",
    "\n",
    "    dcc.Markdown('''\n",
    "    ## Multi-Select Country for visualization\n",
    "    '''),\n",
    "\n",
    "\n",
    "    dcc.Dropdown(\n",
    "        id='country_drop_down',\n",
    "        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],\n",
    "        value=['US', 'Germany','Italy'], # which are pre-selected\n",
    "        multi=True\n",
    "    ),\n",
    "\n",
    "    dcc.Markdown('''\n",
    "        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time\n",
    "        '''),\n",
    "\n",
    "\n",
    "    dcc.Dropdown(\n",
    "    id='doubling_time',\n",
    "    options=[\n",
    "        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},\n",
    "        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},\n",
    "        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},\n",
    "        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},\n",
    "    ],\n",
    "    value='confirmed',\n",
    "    multi=False\n",
    "    ),\n",
    "\n",
    "    dcc.Graph(figure=fig, id='main_window_slope')\n",
    "])\n",
    "\n",
    "\n",
    "\n",
    "@app.callback(\n",
    "    Output('main_window_slope', 'figure'),\n",
    "    [Input('country_drop_down', 'value'),\n",
    "    Input('doubling_time', 'value')])\n",
    "def update_figure(country_list,show_doubling):\n",
    "\n",
    "\n",
    "    if 'doubling_rate' in show_doubling:\n",
    "        my_yaxis={'type':\"log\",\n",
    "               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'\n",
    "              }\n",
    "    else:\n",
    "        my_yaxis={'type':\"log\",\n",
    "                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'\n",
    "              }\n",
    "\n",
    "\n",
    "    traces = []\n",
    "    for each in country_list:\n",
    "\n",
    "        df_plot=df_input_large[df_input_large['country']==each]\n",
    "\n",
    "        if show_doubling=='doubling_rate_filtered':\n",
    "            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()\n",
    "        else:\n",
    "            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()\n",
    "       #print(show_doubling)\n",
    "\n",
    "\n",
    "        traces.append(dict(x=df_plot.date,\n",
    "                                y=df_plot[show_doubling],\n",
    "                                mode='markers+lines',\n",
    "                                opacity=0.9,\n",
    "                                name=each\n",
    "                        )\n",
    "                )\n",
    "\n",
    "    return {\n",
    "            'data': traces,\n",
    "            'layout': dict (\n",
    "                width=1280,\n",
    "                height=720,\n",
    "\n",
    "                xaxis={'title':'Timeline',\n",
    "                        'tickangle':-45,\n",
    "                        'nticks':20,\n",
    "                        'tickfont':dict(size=14,color=\"#7f7f7f\"),\n",
    "                      },\n",
    "\n",
    "                yaxis=my_yaxis\n",
    "        )\n",
    "    }\n",
    "\n",
    "if __name__ == '__main__':\n",
    "\n",
    "    app.run_server(debug=True, use_reloader=False)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}